In [7]:
import sys
!{sys.executable} -m pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 30.0 MB/s eta 0:00:0000:0100:01


In [2]:
!pip install --upgrade transformers

     |████████████████████████████████| 10.4 MB 1.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.0 MB 80.6 MB/s eta 0:00:01
     |████████████████████████████████| 471 kB 9.5 MB/s eta 0:00:01
     |████████████████████████████████| 780 kB 12.8 MB/s eta 0:00:01
You should consider upgrading via the '/home/venkatesh/.pyenv/versions/3.9.7/bin/python3.9 -m pip install --upgrade pip' command.


In [3]:
import transformers
print(transformers.__version__)
print(transformers.__file__)


/home/venkatesh/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.49.0
/home/venkatesh/miniconda3/lib/python3.12/site-packages/transformers/__init__.py


In [4]:
from transformers import TrainingArguments

print(TrainingArguments.__module__)  # Confirm it's from transformers.training_args


transformers.training_args


In [5]:
from transformers import TrainingArguments
import inspect

# Show signature of the __init__ method
print(inspect.signature(TrainingArguments.__init__))


(self, output_dir: Optional[str] = None, overwrite_output_dir: bool = False, do_train: bool = False, do_eval: bool = False, do_predict: bool = False, eval_strategy: Union[transformers.trainer_utils.IntervalStrategy, str] = 'no', prediction_loss_only: bool = False, per_device_train_batch_size: int = 8, per_device_eval_batch_size: int = 8, per_gpu_train_batch_size: Optional[int] = None, per_gpu_eval_batch_size: Optional[int] = None, gradient_accumulation_steps: int = 1, eval_accumulation_steps: Optional[int] = None, eval_delay: Optional[float] = 0, torch_empty_cache_steps: Optional[int] = None, learning_rate: float = 5e-05, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, max_grad_norm: float = 1.0, num_train_epochs: float = 3.0, max_steps: int = -1, lr_scheduler_type: Union[transformers.trainer_utils.SchedulerType, str] = 'linear', lr_scheduler_kwargs: Union[dict, str, NoneType] = <factory>, warmup_ratio: float = 0.0, warmup_ste

In [1]:
from datasets import load_dataset

dataset = load_dataset("casehold/casehold")


/home/venkatesh/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Access each split
train_data = dataset["train"].select(range(3000))
val_data = dataset["validation"].select(range(500))
test_data = dataset["test"].select(range(500))

In [3]:
import pandas as pd

df_train = pd.DataFrame(train_data)
print(df_train.head())
print(len(test_data))

   example_id                                      citing_prompt  \
0           0  Drapeau’s cohorts, the cohort would be a “vict...   
1           1  Colameta used customer information that he too...   
2           2  property tax sale. In reviewing section 6323(b...   
3           3  They also rely on Oswego Laborers’ Local 214 P...   
4           4  did not affect the defendant’s guideline range...   

                                           holding_0  \
0  holding that possession of a pipe bomb is a cr...   
1  recognizing that even if a plaintiff claims ce...   
2  holding that where there is a conflict between...   
3  holding that plaintiff stated a  349 claim whe...   
4  holding that united states v booker 543 us 220...   

                                           holding_1  \
0  holding that bank robbery by force and violenc...   
1  holding that included among trade secrets empl...   
2  holding that specific statutory provisions tak...   
3  holding that plaintiff stat

In [4]:
# Print a single example
print(train_data[0])

{'example_id': 0, 'citing_prompt': "Drapeau’s cohorts, the cohort would be a “victim” of making the bomb. Further, firebombs are inherently dangerous. There is no peaceful purpose for making a bomb. Felony offenses that involve explosives qualify as “violent crimes” for purposes of enhancing the sentences of career offenders. See 18 U.S.C. § 924(e)(2)(B)(ii) (defining a “violent felony” as: “any crime punishable by imprisonment for a term exceeding one year ... that ... involves use of explosives”). Courts have found possession of a'bomb to be a crime of violence based on the lack of a nonviolent purpose for a bomb and the fact that, by its very nature, there is a substantial risk that the bomb would be used against the person or property of another. See United States v. Newman, 125 F.3d 863 (10th Cir.1997) (unpublished) (<HOLDING>); United States v. Dodge, 846 F.Supp. 181,", 'holding_0': 'holding that possession of a pipe bomb is a crime of violence for purposes of 18 usc  3142f1', 'h

In [5]:
##Preprocessing
from transformers import AutoTokenizer

# Load a tokenizer (e.g., BERT)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Preprocessing function
def preprocess_casehold(batch):
    prompts = batch["citing_prompt"]
    options_list = [
        [batch[f"holding_{i}"][j] for i in range(5)]
        for j in range(len(prompts))
    ]

    flat_prompts = [prompt for prompt in prompts for _ in range(5)]
    flat_options = [opt for opts in options_list for opt in opts]

    inputs = tokenizer(
        flat_prompts,
        flat_options,
        truncation=True,
        padding="max_length",
        max_length=256
    )

    # Group back to structure per sample
    num_samples = len(prompts)
    result = {k: [inputs[k][i*5:(i+1)*5] for i in range(num_samples)] for k in inputs}
    result["label"] = [int(l) for l in batch["label"]]
    return result



In [6]:
# input_ids: list of token IDs for each of the 5 pairs

# attention_mask: tells BERT which tokens are real vs padding

# token_type_ids: distinguishes between the prompt and the holding in the input

# label: the correct answer index (0 to 4)

In [7]:
# Apply the function to each dataset split
batch_size = 8 # For preprocessing only

encoded_train = train_data.map(preprocess_casehold, batched=True, batch_size=batch_size)
encoded_val = val_data.map(preprocess_casehold, batched=True, batch_size=batch_size)
encoded_test = test_data.map(preprocess_casehold, batched=True, batch_size=batch_size)

Map: 100%|████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 594.34 examples/s]


In [8]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForMultipleChoice, AutoTokenizer, BertConfig
from torch.optim import AdamW
from datasets import load_dataset
from tqdm import tqdm
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model
# model = BertForMultipleChoice.from_pretrained("bert-base-uncased").to(device)
config = BertConfig.from_pretrained("bert-base-uncased", hidden_dropout_prob=0.2)
model = BertForMultipleChoice.from_pretrained("bert-base-uncased", config=config).to(device)

# Data format (assumes you’ve already preprocessed the dataset!)
encoded_train.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids", "label"])
encoded_val.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids", "label"])
encoded_test.set_format("torch", columns=["input_ids", "attention_mask", "token_type_ids", "label"])




Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# DataLoaders
from transformers import get_scheduler

train_loader = DataLoader(encoded_train, batch_size=8, shuffle=True)
val_loader = DataLoader(encoded_val, batch_size=1)
test_loader = DataLoader(encoded_test, batch_size=1)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.1)

num_epochs = 10
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=int(0.1 * num_training_steps),
    num_training_steps=num_training_steps,
)


In [ ]:
# Accuracy metric
def compute_accuracy(preds, labels):
    return (preds == labels).sum().item() / len(labels)

# Training loop
best_val_acc = 0
patience = 4
patience_counter = 0
for epoch in range(num_epochs):
    print(f"\n Epoch {epoch+1}/{num_epochs}")
    model.train()
    train_loss = 0
    train_correct = 0
    total = 0

    for batch in tqdm(train_loader, desc="Training"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )

        loss = outputs.loss
        logits = outputs.logits

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()   
        lr_scheduler.step()

        train_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        train_correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_accuracy = train_correct / total
    print(f"Training Loss: {train_loss/len(train_loader):.4f}, Accuracy: {train_accuracy:.4f}")

    # Validation
    model.eval()
    val_correct = 0
    total_val = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            labels = batch["label"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )

            preds = torch.argmax(outputs.logits, dim=1)
            val_correct += (preds == labels).sum().item()
            total_val += labels.size(0)

    val_accuracy = val_correct / total_val
    if val_accuracy > best_val_acc:
        best_val_acc = val_accuracy
        patience_counter = 0
    else:
        patience_counter += 1
    
    if patience_counter >= patience:
        print("Early stopping triggered")
        break

    print(f"Validation Accuracy: {val_accuracy:.4f}")




 Epoch 1/10


Training: 100%|███████████████████████████████████████████████████████████████████████| 375/375 [06:16<00:00,  1.00s/it]


Training Loss: 1.2876, Accuracy: 0.4697


Validating: 100%|█████████████████████████████████████████████████████████████████████| 500/500 [00:21<00:00, 22.91it/s]


Validation Accuracy: 0.6760

 Epoch 2/10


...

Training: 100%|███████████████████████████████████████████████████████████████████████| 375/375 [06:36<00:00,  1.06s/it]


Training Loss: 0.3257, Accuracy: 0.8767


Validating: 100%|█████████████████████████████████████████████████████████████████████| 500/500 [00:20<00:00, 24.45it/s]


Validation Accuracy: 0.6920

 Epoch 7/10


Training:  15%|██████████▉                                                             | 57/375 [01:05<07:35,  1.43s/it]

In [12]:
# Final Test Evaluation
model.eval()
test_correct = 0
total_test = 0
all_preds = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Testing"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )

        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().tolist())
        test_correct += (preds == labels).sum().item()
        total_test += labels.size(0)

test_accuracy = test_correct / total_test
print(f"\n🎯 Final Test Accuracy: {test_accuracy:.4f}")

Testing: 100%|████████████████████████████████████████████████████████████████████████| 500/500 [00:20<00:00, 23.87it/s]


🎯 Final Test Accuracy: 0.6700


In [12]:
# from transformers import BertForMultipleChoice, Trainer, TrainingArguments
# from datasets import load_dataset
# import torch

# # 1. Set format for PyTorch tensors
# encoded_train.set_format(type='torch', columns=["input_ids", "attention_mask", "token_type_ids", "label"])
# encoded_val.set_format(type='torch', columns=["input_ids", "attention_mask", "token_type_ids", "label"])
# encoded_test.set_format(type='torch', columns=["input_ids", "attention_mask", "token_type_ids", "label"])

In [13]:
# # 2. Load the model for multiple choice classification
# model = BertForMultipleChoice.from_pretrained("bert-base-uncased")


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# # 3. Define training arguments

# training_args = TrainingArguments(
#     output_dir="./results",
#     report_to="none",
#     eval_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=1,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     logging_dir="./logs"
# )


In [15]:
# # 4. Set up the Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=encoded_train,
#     eval_dataset=encoded_val,
# )

In [16]:
# # 5. Train the model
# trainer.train()

Epoch,Training Loss,Validation Loss
1,0.856000,0.798557
2,0.493400,1.027006
3,0.248400,1.248229


TrainOutput(global_step=3750, training_loss=0.5503912333170573, metrics={'train_runtime': 3432.5178, 'train_samples_per_second': 4.37, 'train_steps_per_second': 1.092, 'total_flos': 9866575987200000.0, 'train_loss': 0.5503912333170573, 'epoch': 3.0})

In [17]:
# # 6. Evaluate on validation set
# trainer.evaluate()

{'eval_loss': 1.2482285499572754,
 'eval_runtime': 67.3056,
 'eval_samples_per_second': 14.858,
 'eval_steps_per_second': 14.858,
 'epoch': 3.0}

In [18]:
# # 7. (Optional) Predict on test set
# predictions = trainer.predict(encoded_test)